# BTC Price Prediction:

## Import all the required packages:

In [1]:
# Necessary imports
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Suppress warnings to keep the output cleanb
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\nihar\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Step 1: Get the BTC price:

In [2]:
# Download Bitcoin price data from Yahoo Finance (data available as far back as 2014-09-16)
btc_df = yf.download('BTC-USD', start='2014-09-16', end=datetime.now(), interval='1d')

# Display the first few rows of the dataset
btc_df.tail(20)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-06-19,65146.660156,65695.351562,64693.300781,64960.296875,64960.296875,21103423504
2024-06-20,64960.296875,66438.960938,64547.847656,64828.656250,64828.656250,25641109124
2024-06-21,64837.988281,65007.546875,63378.894531,64096.199219,64096.199219,26188171739
2024-06-22,64113.863281,64475.468750,63929.757812,64252.578125,64252.578125,9858198793
2024-06-23,64248.964844,64491.703125,63180.796875,63180.796875,63180.796875,11170471802
2024-06-24,63173.351562,63292.527344,58601.699219,60277.414062,60277.414062,43152133651
2024-06-25,60266.281250,62258.261719,60239.750000,61804.640625,61804.640625,29201215431
2024-06-26,61789.675781,62434.136719,60695.187500,60811.277344,60811.277344,22506003064
2024-06-27,60811.226562,62293.863281,60585.332031,61604.800781,61604.800781,21231745045


# Calculate Monday to Monday returns:

In [3]:
# Function to calculate 7-day ROI from Monday to Monday to follow tournament's Monday to Monday forecast schedule.

def calculate_monday_to_monday_roi(btc_df):
    monday_close = btc_df[btc_df.index.weekday == 0]['Close']
    next_monday_close = monday_close.shift(-1)  # Shift by 1 week
    weekly_returns = pd.DataFrame({'Monday_Close': monday_close, 'Next_Monday_Close': next_monday_close})
    weekly_returns.dropna(inplace=True)
    weekly_returns['7_Day_ROI'] = (weekly_returns['Next_Monday_Close'] - weekly_returns['Monday_Close']) / weekly_returns['Monday_Close'] * 100
    weekly_returns = weekly_returns[['7_Day_ROI']]  # Only keep the 7_Day_ROI
    return weekly_returns

# Get the weekly BTC ROI data
weekly_btc_df = calculate_monday_to_monday_roi(btc_df)
weekly_btc_df

,7_Day_ROI
Date,
2014-09-22,-6.635550
2014-09-29,-12.088412
2014-10-06,18.278954
2014-10-13,-1.938711
2014-10-20,-7.798453
...,...
2024-06-03,1.028272
2024-06-10,-4.347411
2024-06-17,-9.344044


## Step 2: Create the Exponential Smoothing Model:

In [4]:
def weekly_roi_model(train):
    model = ExponentialSmoothing(
        train['7_Day_ROI'], 
        seasonal='add', 
        trend='add', 
        damped_trend=True, 
        seasonal_periods=48, 
        initialization_method='legacy-heuristic'
    ).fit()
    return model


## Write the forecasting function:

In [5]:
# Define rolling forecast function for weekly ROI
def rolling_forecast(model_func, data, forecast_steps, rolling_periods):
    forecasted_rois = []
    actual_rois = []
    initial_train_size = len(data) - rolling_periods
    train_set = data[:initial_train_size]
    
    for i in range(rolling_periods):
        test_set_start = initial_train_size + i
        test_set_end = test_set_start + forecast_steps
        
        # Train model on current training set
        model = model_func(train_set)
        
        # Forecast the next week
        forecast = model.forecast(steps=forecast_steps)
        forecasted_roi = forecast.iloc[-1]  # Take the last value of the forecast
        
        # Actual ROI
        actual_roi = data['7_Day_ROI'].iloc[test_set_start]
        
        # Store ROIs
        forecasted_rois.append(forecasted_roi)
        actual_rois.append(actual_roi)
        
        # Update training set with actual data of the forecasted week
        train_set = data[:test_set_end]
    
    # Calculate overall MAE
    mae = mean_absolute_error(actual_rois, forecasted_rois)
    return mae, forecasted_rois, actual_rois

# Backtesting parameters
forecast_steps = 1  # 1 week
rolling_periods = 4  # 4 weeks

# Perform rolling forecast
mae, forecasted_rois, actual_rois = rolling_forecast(weekly_roi_model, weekly_btc_df, forecast_steps, rolling_periods)

In [6]:
print(f'MAE: {mae:.2f}%')
print(f'Forecasted ROIs: {forecasted_rois}')
print(f'Actual ROIs: {actual_rois}')

MAE: 6.29%
Forecasted ROIs: [-0.8779195592241669, -5.047856193585729, -0.4530181279708936, 2.882314677692075]
Actual ROIs: [-4.347410731826615, -9.344044326016554, 4.2711958472214855, -9.779934962520759]


In [7]:
# Train the model on the entire dataset
model_dhr = weekly_roi_model(weekly_btc_df)

# Forecast the ROI for the week from July 8th to July 15th
forecasted_roi = model_dhr.forecast(steps=1)[0]
print(f'Forecasted ROI for the week from July 8th to July 15th: {forecasted_roi:.3f}')


Forecasted ROI for the week from July 8th to July 15th: 5.722
